In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
from tensorflow.keras.preprocessing import image

In [5]:
dataset_path = "C:\kalyan\College Projects\Blood Group\dataset_blood_group"  # path to your folder with blood groups

# ImageDataGenerator with train-validation split
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,   # 80-20 split
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Found 4803 images belonging to 8 classes.
Found 1197 images belonging to 8 classes.


In [6]:
num_classes = len(train_gen.class_indices)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 128)      0

In [7]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=40  # increase if dataset is large
)


Epoch 1/40
151/151 [==============================] - 213s 1s/step - loss: 1.9483 - accuracy: 0.2288 - val_loss: 1.6783 - val_accuracy: 0.3275
Epoch 2/40
151/151 [==============================] - 39s 256ms/step - loss: 1.6269 - accuracy: 0.3458 - val_loss: 1.4587 - val_accuracy: 0.4344
Epoch 3/40
151/151 [==============================] - 35s 233ms/step - loss: 1.5025 - accuracy: 0.3950 - val_loss: 1.3231 - val_accuracy: 0.4662
Epoch 4/40
151/151 [==============================] - 35s 229ms/step - loss: 1.4659 - accuracy: 0.3960 - val_loss: 1.2813 - val_accuracy: 0.4662
Epoch 5/40
151/151 [==============================] - 33s 220ms/step - loss: 1.3446 - accuracy: 0.4495 - val_loss: 1.3476 - val_accuracy: 0.4495
Epoch 6/40
151/151 [==============================] - 34s 224ms/step - loss: 1.3145 - accuracy: 0.4445 - val_loss: 1.2212 - val_accuracy: 0.4987
Epoch 7/40
151/151 [==============================] - 32s 213ms/step - loss: 1.3002 - accuracy: 0.4678 - val_loss: 1.1953 - val_accu

In [79]:
val_loss, val_acc = model.evaluate(val_gen)
print(f"Validation Accuracy: {val_acc*100:.2f}%")

38/38 [==============================] - 4s 100ms/step - loss: 0.5682 - accuracy: 0.7870
Validation Accuracy: 78.70%


In [75]:
import numpy as np
img_path = r"C:\kalyan\College Projects\Blood Group\dataset_blood_group\O+\cluster_6_724.BMP"

img = image.load_img(img_path, target_size=(128, 128))

# Convert to array
img_array = image.img_to_array(img)

# Normalize (if you used ImageDataGenerator(rescale=1./255))
img_array = img_array / 255.0

# Expand dims to add batch axis: (1, 128, 128, 3)
img_array = np.expand_dims(img_array, axis=0)

In [76]:
pred = model.predict(img_array)
print("Raw prediction vector:", pred)

1/1 [==============================] - 0s 25ms/step
Raw prediction vector: [[2.1908828e-03 9.5255598e-03 2.8651975e-08 3.5466803e-07 6.5685777e-09
  3.1232021e-03 9.8515815e-01 1.7217974e-06]]


In [77]:
class_labels = ['A+', 'A-', 'AB+', 'AB-', 'B+', 'B-', 'O+', 'O-']

# Get predicted index
predicted_index = np.argmax(pred, axis=1)[0]

# Get class name
predicted_class = class_labels[predicted_index]
confidence = pred[0][predicted_index]

print(f"Predicted class: {predicted_class}, Confidence: {confidence:.2f}")


Predicted class: O+, Confidence: 0.99


In [80]:
model.save("cnn_model_sairam.h5")